In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib tk

In [3]:
import os, os.path, sys

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\Y"

In [5]:
dir = os.listdir(path)

Ucitavanje fajlova

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [8]:
train_files_names = file_names[:int(percent_train*len(file_names))]
test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [9]:
y_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\yCmd.txt",header=None).astype('int'))

Konkatenacija fajlova koji se posle pravilno rasporedjuju na adekvatne train/test skupove

In [10]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in test_file_names], axis=0)

Prethodno konkatenizovani podaci se adekvatno dele kako bi se izbeglo fiktivno stanje (stanje pri kraju rada i pocetku novog ako su konkatenizovani podaci)

In [ ]:
for j in range(len(train_files_names)):
    exec(f"koord_trening_{j} = []")
    exec(f"razlika_trening_{j} = []")

    for i in range(61000):
        exec(f"koord_trening_{j}.append(train_[j*61000+i])")
        exec(f"razlika_trening_{j}.append(y_command[i] - train_[j*61000+i])")
    exec(f'koord_trening_{j} = np.asarray(koord_trening_{j})') 
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')  

In [ ]:
for j in range(len(test_file_names)):
    exec(f"koord_testing_{j} = []")
    exec(f"razlika_test_{j} = []")
    for i in range(61000):
        exec(f"koord_testing_{j}.append(train_[j*61000+i])")
        exec(f"razlika_test_{j}.append(y_command[i] - test_[j*61000+i])")
    exec(f'koord_testing_{j} = np.asarray(koord_testing_{j})')
    exec(f'razlika_test_{j} = np.asarray(razlika_test_{j})') 

Funkcija koja deli podatke na "sliding window"

In [17]:
def create_timeseries_split(data, time_steps):

    # Get the number of samples in the data
    num_samples = len(data) - time_steps

    # Create empty arrays to store features and target values
    x_train = np.zeros((num_samples, time_steps, 1))
    y_train = np.zeros((num_samples, 1))

    # Loop through the data and create features and target values
    for i in range(num_samples):
        # Extract a slice of data for the current feature
        x_train[i] = data[i:i+time_steps, :]

        # The target value is the next value after the feature
        y_train[i] = data[i+time_steps, 0]

    return x_train, y_train

Velicina buffera

In [21]:
time_steps = 8

Deljenje podataka na train i test skup, takodje delimo ciljnu promenljivu y na odstupanja dok podatke za treniranje train_x delimo na koordinate

In [22]:
for i in range(len(train_files_names)):
    exec(f'train_koord_X_{i},_ = create_timeseries_split(koord_trening_{i}, time_steps)')

In [23]:
for i in range(len(test_file_names)):
    exec(f'test_koord_X_{i}, _ = create_timeseries_split(koord_testing_{i}, time_steps)')

Sada pravimo y skup za odstupanja

In [24]:
for i in range(len(train_files_names)):
    exec(f'_, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')

In [25]:
for i in range(len(test_file_names)):
    exec(f'_, test_y_{i} = create_timeseries_split(razlika_test_{i}, time_steps)')

konkatenacija za X

In [26]:
train_X = np.asarray(train_koord_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_koord_X_{i}, axis=0)')

konkatenacije za y

In [27]:
train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')

konk za X

In [32]:
test_X = np.asarray(train_koord_X_0)
for i in range(1,len(test_file_names)):
    exec(f'test_X = np.append(test_X, test_koord_X_{i}, axis=0)')

konk za y

In [36]:
test_y = np.asarray(test_y_0)
for i in range(1,len(test_file_names)):
    exec(f'test_y = np.append(test_y, test_y_{i}, axis=0)')

In [37]:
input_shape = (time_steps,1)

In [38]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(GRU(units = 16, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [39]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/10

5718/5718 - 26s - loss: 1086.0851 - 26s/epoch - 5ms/step
Epoch 2/10
5718/5718 - 28s - loss: 57.3289 - 28s/epoch - 5ms/step
Epoch 3/10
5718/5718 - 29s - loss: 83.1188 - 29s/epoch - 5ms/step
Epoch 4/10
5718/5718 - 30s - loss: 54.0706 - 30s/epoch - 5ms/step
Epoch 5/10
5718/5718 - 29s - loss: 57.6154 - 29s/epoch - 5ms/step
Epoch 6/10
5718/5718 - 30s - loss: 52.5054 - 30s/epoch - 5ms/step
Epoch 7/10
5718/5718 - 29s - loss: 57.2826 - 29s/epoch - 5ms/step
Epoch 8/10
5718/5718 - 30s - loss: 52.8034 - 30s/epoch - 5ms/step
Epoch 9/10
5718/5718 - 31s - loss: 51.3773 - 31s/epoch - 5ms/step
Epoch 10/10
5718/5718 - 38s - loss: 50.8868 - 38s/epoch - 7ms/step


In [50]:
train_scores = modelMSE.evaluate(test_X, test_y)

11436/11436 [==============================] - 19s 2ms/step - loss: 17.5234


In [40]:
#train_predict_MSE = modelMSE.predict(train_X)
test_predict_MSE = modelMSE.predict(test_X)

11436/11436 [==============================] - 47s 4ms/step


In [41]:
modelMSE.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 16)                912       
                                                                 
 dense (Dense)               (None, 8)                 136       
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1209 (4.72 KB)
Trainable params: 1209 (4.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Ucitavamo standardne devijacije i srednje vrednosti

In [42]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [43]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

In [44]:
mean_1std = 0
for j in range(len(test_file_names)):
    for i in range(int(len(test_predict_MSE)/len(test_file_names))):
        if sr_vrednost[i+time_steps] - std_dev[i+time_steps] <= test_predict_MSE[int(len(test_predict_MSE)/len(test_file_names))*j + i] <= sr_vrednost[i+time_steps] + std_dev[i+time_steps]:
            mean_1std += 1
        #else:
            #print(sr_vrednost[i+8], std_dev[i+8], test_predict_MSE[60992*j + i], 'index:',j*60992 + i)
mean_1std/len(test_predict_MSE)

0.33227855019237496

mean 3 sigma

In [45]:
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%61000 == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.5712743747813921

pravljenje min/max koord, posle se koriste za plot

In [37]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for i in range(61000):    
    for j in range(len(train_files_names)):
        exec(f'current_file = razlika_trening_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

cuvamo/ucitavamo model

# NA TEST SKUPU - plotovi trajektorija

test_ = realizovana trajektorija

test_predict_MSE = predvidjena RAZLIKA koordinata od x_command

razlika_test_{} = realizovana RAZLIKA od test podataka od x_command - CELA TRAJEKTORIJA

test_y_{} = REALIZOVANA y koordinata od razlika_test_{}

X_COMMAND - TEST_ = ODSTUPANJE/RAZLIKA_TEST_

X_COMMAND[TIME_STEPS] + test_predict_MSE[0] = PRVU KOORDINATU TRAJEKTORIJE PREDVIDJENE VREDNOSTI

In [33]:
for j in range(len(test_file_names)):
    exec(f'predvidjena_trajektorija_{j} = []')
    for i in range(61000 - time_steps):
        exec(f'predvidjena_trajektorija_{j}.append(x_command[i + time_steps] + test_predict_MSE[j*(61000-time_steps) + i])')

In [35]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_0,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_0,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [47]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_0,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [64]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_1,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_1,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [51]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_1,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [52]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [54]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [56]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [58]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [59]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [60]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [62]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [63]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

# NA TEST SKUPU - min/max plotovi

MIN MAX TRAJEKTORIJE

In [38]:
min_trajektorija = []
max_trajektorija = []
for i in range(61000):
    min_trajektorija.append(x_command[i] + min_kord[i])
    max_trajektorija.append(x_command[i] + max_kord[i])

In [39]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [212]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_1)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_1[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_1[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_1[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [213]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_2)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_2[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_2[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_2[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [41]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_3)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_3[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_3[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_3[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [43]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_4)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_4[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_4[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_4[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [45]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()